# Import

In [1]:
import csv
import os
import sys

import numpy as np
import pandas as pd
import umap
from scipy.sparse.csgraph import connected_components
from sklearn.decomposition import PCA
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config 
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
s3 = S3Manager()

In [6]:
data_type="20News"

In [7]:
transformer_model = "sentence-transformers/all-MiniLM-L6-v2"

# Read data

In [8]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

['Preprocessing/20News/master.csv']


In [9]:
df = pd.read_csv(df_path, index_col=0)

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

In [ ]:
with open(labels_path, mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [ ]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

df = pd.read_csv(df_path, index_col=0)

labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

with open(labels_path, mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [ ]:
s3.download(f"Vectorize/")

In [27]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]
vector_dims = config["vectorize"]["sentenceBERT"]["dims"]

# Dimension Depression

In [28]:
vector_path = (
    f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector"
)

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = PCA(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)

        np.save(
            make_filepath(f"{vector_path}/pca/{vector_dim}/{model_num}.npy"),
            reduced_vectors,
        )

  0%|          | 0/10 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../temporary/Vectorize/20News/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/raw/0.npy'

In [ ]:
vector_path = f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector"

for vector_dim in tqdm(vector_dims):
    for model_num in range(max_model_num):
        vectors = np.load(
            f"{vector_path}/raw/{model_num}.npy",
        )
        reduced_vectors = umap.UMAP(
            n_components=vector_dim, random_state=model_num
        ).fit_transform(vectors)
        np.save(
            make_filepath(f"{vector_path}/umap/{vector_dim}/{model_num}.npy"),
            reduced_vectors,
        )

## upload file

In [ ]:
s3.upload(
    f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector", 
    f"Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector"
)

In [ ]:
s3.delete_local_all()